# LA NIÑA RAINFALL EVENT ANALYSIS: AUCKLAND, NEW ZEALAND JANUARY 2023
### Data Science using Python

###### Ivana J. Mc Donald

2023/2024 Data Analyst Project

**************************************************************************************************

### Rainfall Dataset
National Institute of Water and Atmospheric Research (NIWA). (2023). "CliFlo: The National Climate Database" Retrieved from [ https://cliflo.niwa.co.nz/pls/niwp/wgenf.genform1 ].

**************************************************************************************************

### DataFrame Descriptions
###### rainfall_data_df
_Section 2.1.5 displays 'rainfall_data_df'_

rainfall_data_df is a historical record of daily rainfall in Auckland from July 1, 1962, to June 30, 2023, with columns for Station, Date, and Rainfall.
###### year_month_df
_Section 3.1.6 displays 'year_month_df'_

year_month_df contains data with two columns: 'Year and Month' and 'Monthly Rainfall (mm)', showing the total rainfall recorded for each month over multiple years.
###### monthly_rainfall_df
_Section 3.2.17 displays 'monthly_rainfall_df'_

monthly_rainfall_df is a dataset showing the total monthly and annual rainfall in corresponding years. It contains columns for Year, January to December, and Annual Rainfall.
###### monthly_stats_df
_Section 3.3.8 displays 'monthly_stats_df'_

monthly_stats_df presents monthly statistics (Min, Max, Average, Std Deviation, and Mean) calculated from data recorded between July 1, 1962, and June 30, 2023, for each month.
###### seasonal_rainfall_df
_Section 3.4.8 displays 'seasonal_rainfall_df'_

seasonal_rainfall_df shows the total rainfall for each season (Summer, Autumn, Winter, and Spring) corresponding to the years listed in the 'Year' column.
###### seasonal_stats_df
_Section 3.5.8 displays 'seasonal_stats_df'_

seasonal_stats_df presents seasonal statistics (Min, Max, Average, Std Deviation, and Mean) calculated from data recorded between July 1, 1962, and June 30, 2023, for each season.
###### gumbel_est_calculations_df
_Section 4.1.11 displays 'gumbel_est_calculations_df'_

gumbel_est_calculations_df contains Gumbel estimates, return periods, and relevant variables (Max One-Day Rainfall, Observations, Rank, Exceedance, Non-Exceedance, Gumbel Est. Return Period, Reduced Variable) for rainfall data analysis. 
###### gumbel_variables_df
_Section 4.2.5 displays 'gumbel_variables_df'_

gumbel_variables_df shows the variables, their corresponding equations, and the calculated values used in Gumbel Estimate calculations. Useful for understanding the process and results of the analysis. 
###### gumbel_est_df
_Section 4.3.6 displays 'gumbel_est_df'_

gumbel_est_df contains Gumbel Estimate results for various return periods, based on corresponding Gumbel Estimate Return Period and Reduced Variable values. Useful for analyzing extreme event probabilities.
###### january_2023_stats_df
_Section 4.4.6 displays 'january_2023_stats_df'_

january_2023_stats_df presents a comparison of statistical rainfall data for January between 1963 and 2022 and January 2023. It includes Min, Max, Average, Mean, and Standard Deviation values.

**************************************************************************************************
**************************************************************************************************

# 1. Import Modules & Libraries

In [1]:
import math
import statistics
import time
import warnings
import numpy as np
import openpyxl
import pandas as pd
import plotly.graph_objects as go
import plotly.offline as pyo
from IPython.display import Image
from openpyxl import load_workbook
from openpyxl.chart import BarChart, LineChart
from openpyxl.chart import Reference
from openpyxl.styles import PatternFill
warnings.simplefilter('ignore') # filter some warning messages
import dataframe_image as dfi
import os

**************************************************************************************************
**************************************************************************************************

# 2. Data Manipulation

## 2.1 Create 'rainfall_data_df' Dataframe

### 2.1.1 Read 'original_auckland_rainfall.xlsx' Excel File

#### Read Excel File

In [2]:
rainfall_data_df = pd.read_excel('original_auckland_rainfall.xlsx')

### 2.1.2 Drop Columns

In [3]:
rainfall_data_df = rainfall_data_df.drop(columns=['Time(NZST)', 'Deficit(mm)', 'Runoff(mm)', 'Rel', 'Orig', 'SofG', 'Rel_1', 'Orig_2', 'Period(Hrs)', 'Freq'])

### 2.1.3 Change 'Date (NZST)' Format and Datatype

#### Change Datatype of 'rainfall_data_df['Date(NZST)']'

In [4]:
rainfall_data_df['Date(NZST)'] = rainfall_data_df['Date(NZST)'].astype(str)

#### Insert 'Date (NZST)' Column

In [5]:
rain_dates_list = []
for i in range(len(rainfall_data_df)):
    rain_year_list = rainfall_data_df['Date(NZST)'].iloc[i][:4]
    rain_month_list = rainfall_data_df['Date(NZST)'].iloc[i][4:6]
    rain_day_list = rainfall_data_df['Date(NZST)'].iloc[i][6:8]
    rain_dates_list.append(rain_year_list + '/' + rain_month_list + '/' + rain_day_list)

In [6]:
rainfall_data_df.insert(1, 'Date (NZST)', rain_dates_list)

#### Change Datatype of 'rainfall_data_df['Date (NZST)']'

In [7]:
rainfall_data_df['Date (NZST)'] = pd.to_datetime(rainfall_data_df['Date (NZST)'])

#### Drop 'Date(NZST)'Column

In [8]:
rainfall_data_df = rainfall_data_df.drop(columns=['Date(NZST)'])

### 2.1.4 Change Column Names

In [9]:
rainfall_data_df.rename(columns = {'Date (NZST)':'Date', 'Amount(mm)':'Rainfall (mm/d)', 'Deficit(mm)':'Deficit (mm/d)', 'Runoff(mm)':'Runoff (mm/d)'}, inplace = True)

### 2.1.5 Display 'rainfall_data_df' Dataframe

In [10]:
rainfall_data_df

,Station,Date,Rainfall (mm/d)
0,Auckland Aero,1962-07-01,0.0
1,Auckland Aero,1962-07-02,0.0
2,Auckland Aero,1962-07-03,7.9
3,Auckland Aero,1962-07-04,0.0
4,Auckland Aero,1962-07-05,9.7
...,...,...,...
22076,Auckland Aero,2023-06-25,1.4
22077,Auckland Aero,2023-06-26,0.2
22078,Auckland Aero,2023-06-27,15.2
22079,Auckland Aero,2023-06-28,3.2


**************************************************************************************************
**************************************************************************************************

# 3. Data Visualisation: Typical Rainfall Patterns in Auckland, New Zealand

## 3.1 Create 'year_month_df' Dataframe

### 3.1.1 Create 'yyyy_mm_daily_df' Dataframe

In [11]:
yyyy_mm_daily_df = pd.DataFrame()

### 3.1.2 Insert 'Year and Month' Column

In [12]:
rainfall_date = pd.Series(rainfall_data_df['Date'], dtype="string")

In [13]:
year_month_list = []
for i in range(len(rainfall_date)):
    rainfall_yyyy_mm = rainfall_date[i][:7]
    year_month_list.append(rainfall_yyyy_mm)

In [14]:
yyyy_mm_daily_df.insert(0, 'Year and Month', year_month_list)

In [15]:
yyyy_mm_daily_df.insert(1, 'Rainfall (mm/d)', rainfall_data_df['Rainfall (mm/d)'])

### 3.1.3 Create 'year_month_df' Dataframe

In [16]:
year_month_df = pd.DataFrame()

### 3.1.4 Insert 'Year and Month' Column

In [17]:
yyyy_mm = yyyy_mm_daily_df['Year and Month'].drop_duplicates()

In [18]:
yyyy_mm_list = ['1962-01', '1962-02', '1962-03', '1962-04', '1962-05', '1962-06']

In [19]:
for i in range(len(yyyy_mm)):
    yyyy_mm_list.append(yyyy_mm.iloc[i])

In [20]:
yyyy_mm_append = ['2023-07', '2023-08', '2023-09', '2023-10', '2023-11', '2023-12']
for i in range(len(yyyy_mm_append)):
    yyyy_mm_list.append(yyyy_mm_append[i])

In [21]:
year_month_df.insert(0, 'Year and Month', yyyy_mm_list)

### 3.1.5 Insert 'Rainfall (mm)' Column

In [22]:
monthly_sum = yyyy_mm_daily_df.groupby('Year and Month')['Rainfall (mm/d)'].sum()

In [23]:
monthly_sum_list = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]

In [24]:
for i in range(len(monthly_sum)):
    monthly_sum_list.append(monthly_sum.iloc[i])

In [25]:
monthly_sum_append = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
for i in range(len(monthly_sum_append)):
    monthly_sum_list.append(monthly_sum_append[i])

In [26]:
year_month_df.insert(1, 'Monthly Rainfall (mm)', monthly_sum_list)

### 3.1.6 Display 'year_month_df' Dataframe

In [27]:
year_month_df.head(24)

,Year and Month,Monthly Rainfall (mm)
0,1962-01,NaN
1,1962-02,NaN
2,1962-03,NaN
3,1962-04,NaN
4,1962-05,NaN
5,1962-06,NaN
6,1962-07,158.0
7,1962-08,109.5
8,1962-09,99.8
9,1962-10,182.7


**************************************************************************************************

## 3.2 Create 'monthly_rainfall_df' Dataframe

### 3.2.1 Create 'monthly_rainfall_df' Dataframe

In [28]:
monthly_rainfall_df = pd.DataFrame()

### 3.2.2 Insert 'Year' Column

In [29]:
end_year = rainfall_data_df['Date'].max().year
start_year = rainfall_data_df['Date'].min().year
n_year = int(end_year) - int(start_year) + 1

In [30]:
year_list = []
for i in range(n_year):
    year_list.append(start_year + i)

In [31]:
monthly_rainfall_df.insert(0, 'Year', year_list)

### 3.2.3 Insert 'January' Column

In [32]:
january = []
for i in range(len(year_month_df)):
    if year_month_df['Year and Month'].iloc[i][5:]  == '01':
        m_date = year_month_df['Year and Month'].iloc[i]
        month_rain = (year_month_df['Monthly Rainfall (mm)'][year_month_df['Year and Month'] == m_date])
        month_rain = float(month_rain)
        january.append(month_rain)

In [33]:
monthly_rainfall_df.insert(1, 'January', january)

### 3.2.4 Insert 'February' Column

In [34]:
february = []
for i in range(len(year_month_df)):
    if year_month_df['Year and Month'].iloc[i][5:]  == '02':
        m_date = year_month_df['Year and Month'].iloc[i]
        month_rain = (year_month_df['Monthly Rainfall (mm)'][year_month_df['Year and Month'] == m_date])
        month_rain = float(month_rain)
        february.append(month_rain)

In [35]:
monthly_rainfall_df.insert(2, 'February', february)

### 3.2.5 Insert 'March' Column

In [36]:
march = []
for i in range(len(year_month_df)):
    if year_month_df['Year and Month'].iloc[i][5:]  == '03':
        m_date = year_month_df['Year and Month'].iloc[i]
        month_rain = (year_month_df['Monthly Rainfall (mm)'][year_month_df['Year and Month'] == m_date])
        month_rain = float(month_rain)
        march.append(month_rain)

In [37]:
monthly_rainfall_df.insert(3, 'March', march)

### 3.2.6 Insert 'April' Column

In [38]:
april = []
for i in range(len(year_month_df)):
    if year_month_df['Year and Month'].iloc[i][5:]  == '04':
        m_date = year_month_df['Year and Month'].iloc[i]
        month_rain = (year_month_df['Monthly Rainfall (mm)'][year_month_df['Year and Month'] == m_date])
        month_rain = float(month_rain)
        april.append(month_rain)

In [39]:
monthly_rainfall_df.insert(4, 'April', april)

### 3.2.7 Insert 'May' Column

In [40]:
may = []
for i in range(len(year_month_df)):
    if year_month_df['Year and Month'].iloc[i][5:]  == '05':
        m_date = year_month_df['Year and Month'].iloc[i]
        month_rain = (year_month_df['Monthly Rainfall (mm)'][year_month_df['Year and Month'] == m_date])
        month_rain = float(month_rain)
        may.append(month_rain)

In [41]:
monthly_rainfall_df.insert(5, 'May', may)

### 3.2.8 Insert 'June' Column

In [42]:
june = []
for i in range(len(year_month_df)):
    if year_month_df['Year and Month'].iloc[i][5:]  == '06':
        m_date = year_month_df['Year and Month'].iloc[i]
        month_rain = (year_month_df['Monthly Rainfall (mm)'][year_month_df['Year and Month'] == m_date])
        month_rain = float(month_rain)
        june.append(month_rain)

In [43]:
monthly_rainfall_df.insert(6, 'June', june)

### 3.2.9 Insert 'July' Column

In [44]:
july = []
for i in range(len(year_month_df)):
    if year_month_df['Year and Month'].iloc[i][5:]  == '07':
        m_date = year_month_df['Year and Month'].iloc[i]
        month_rain = (year_month_df['Monthly Rainfall (mm)'][year_month_df['Year and Month'] == m_date])
        month_rain = float(month_rain)
        july.append(month_rain)

In [45]:
monthly_rainfall_df.insert(7, 'July', july)

### 3.2.10 Insert 'August' Column

In [46]:
august = []
for i in range(len(year_month_df)):
    if year_month_df['Year and Month'].iloc[i][5:]  == '08':
        m_date = year_month_df['Year and Month'].iloc[i]
        month_rain = (year_month_df['Monthly Rainfall (mm)'][year_month_df['Year and Month'] == m_date])
        month_rain = float(month_rain)
        august.append(month_rain)

In [47]:
monthly_rainfall_df.insert(8, 'August', august)

### 3.2.11 Insert 'September' Column

In [48]:
september = []
for i in range(len(year_month_df)):
    if year_month_df['Year and Month'].iloc[i][5:]  == '09':
        m_date = year_month_df['Year and Month'].iloc[i]
        month_rain = (year_month_df['Monthly Rainfall (mm)'][year_month_df['Year and Month'] == m_date])
        month_rain = float(month_rain)
        september.append(month_rain)

In [49]:
monthly_rainfall_df.insert(9, 'September', september)

### 3.2.12 Insert 'October' Column

In [50]:
october = []
for i in range(len(year_month_df)):
    if year_month_df['Year and Month'].iloc[i][5:]  == '10':
        m_date = year_month_df['Year and Month'].iloc[i]
        month_rain = (year_month_df['Monthly Rainfall (mm)'][year_month_df['Year and Month'] == m_date])
        month_rain = float(month_rain)
        october.append(month_rain)

In [51]:
monthly_rainfall_df.insert(10, 'October', october)

### 3.2.13 Insert 'November' Column

In [52]:
november = []
for i in range(len(year_month_df)):
    if year_month_df['Year and Month'].iloc[i][5:]  == '11':
        m_date = year_month_df['Year and Month'].iloc[i]
        month_rain = (year_month_df['Monthly Rainfall (mm)'][year_month_df['Year and Month'] == m_date])
        month_rain = float(month_rain)
        november.append(month_rain)

In [53]:
monthly_rainfall_df.insert(11, 'November', november)

### 3.2.14 Insert 'December' Column

In [54]:
december = []
for i in range(len(year_month_df)):
    if year_month_df['Year and Month'].iloc[i][5:]  == '12':
        m_date = year_month_df['Year and Month'].iloc[i]
        month_rain = (year_month_df['Monthly Rainfall (mm)'][year_month_df['Year and Month'] == m_date])
        month_rain = float(month_rain)
        december.append(month_rain)

In [55]:
monthly_rainfall_df.insert(12, 'December', december)

### 3.2.15 Insert 'Annual Rainfall' Column

In [56]:
total_rain_year = []
for i in range(len(monthly_rainfall_df)):
    yearly_sum_rain = (monthly_rainfall_df.iloc[i].sum())-(monthly_rainfall_df['Year'].iloc[i])
    total_rain_year.append(yearly_sum_rain)

In [57]:
monthly_rainfall_df.insert(13, 'Annual Rainfall', total_rain_year)

### 3.2.16 Change NaN Values

In [58]:
monthly_rainfall_df = monthly_rainfall_df.fillna('')

### 3.2.17 Display 'monthly_rainfall_df' Dataframe

In [59]:
#No data during Jan - June 1962 and July - December 2023. For accurate calculations, change Summer & Autmn 1962 and Winter & Spring values.
monthly_rainfall_df['Annual Rainfall'].iloc[0] = 'n/a'
monthly_rainfall_df['Annual Rainfall'].iloc[61] = 'n/a'

In [60]:
monthly_rainfall_df

,Year,January,February,March,April,May,June,July,August,September,October,November,December,Annual Rainfall
0,1962,,,,,,,158.0,109.5,99.8,182.7,117.0,97.2,n/a
1,1963,58.3,92.6,77.3,101.7,68.0,122.6,124.9,67.3,107.5,11.0,71.9,80.3,983.4
2,1964,64.3,32.5,104.6,21.1,128.6,113.3,175.5,161.2,112.7,142.0,49.6,127.5,1232.9
3,1965,125.2,144.3,88.1,93.3,103.7,137.8,144.1,220.4,39.3,64.4,98.2,92.8,1351.6
4,1966,109.2,144.9,133.5,77.6,135.6,129.3,106.9,93.2,93.0,50.1,133.6,89.0,1295.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,2019,31.6,20.6,40.8,79.6,45.2,103.4,135.6,199.8,122.8,72.2,39.4,51.0,942.0
58,2020,7.8,14.4,40.2,32.8,97.6,168.8,91.6,121.2,43.8,23.8,107.8,44.0,793.8
59,2021,40.6,49.2,37.0,77.8,62.8,106.6,87.6,89.8,106.4,110.4,37.8,106.2,912.2
60,2022,8.8,63.2,121.4,31.4,63.8,158.4,256.0,158.6,95.0,125.6,157.4,114.2,1353.8


**************************************************************************************************

## 3.3 Create 'monthly_stats_df' Dataframe

### 3.3.1 Create 'monthly_stats_df' Dataframe

In [61]:
monthly_stats_df = pd.DataFrame()

### 3.3.2 Insert 'Month' Column

In [62]:
month_index = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

In [63]:
monthly_stats_df.insert(0, 'Month', month_index)

### 3.3.3 Insert 'Min Monthly Rainfall' Column

In [64]:
monthly_len = float(len(monthly_rainfall_df))

In [65]:
jan0 = monthly_rainfall_df['January'].iloc[1:62].min()
feb0 = monthly_rainfall_df['February'].iloc[1:62].min()
mch0 = monthly_rainfall_df['March'].iloc[1:62].min()
apr0 = monthly_rainfall_df['April'].iloc[1:62].min()
may0 = monthly_rainfall_df['May'].iloc[1:62].min()
jun0 = monthly_rainfall_df['June'].iloc[1:62].min()
jul0 = monthly_rainfall_df['July'].iloc[0:61].min()
aug0 = monthly_rainfall_df['August'].iloc[0:61].min()
sep0 = monthly_rainfall_df['September'].iloc[0:61].min()
octb0 = monthly_rainfall_df['October'].iloc[0:61].min()
nov0 = monthly_rainfall_df['November'].iloc[0:61].min()
dec0 = monthly_rainfall_df['December'].iloc[0:61].min()

In [66]:
min_one_day = [jan0, feb0, mch0, apr0, may0, jun0, jul0, aug0, sep0, octb0, nov0, dec0]

In [67]:
monthly_stats_df.insert(1, 'Min Monthly Rainfall', min_one_day)

### 3.3.4 Insert 'Max Monthly Rainfall' Column

In [68]:
jan1 = monthly_rainfall_df['January'].iloc[1:62].max()
feb1 = monthly_rainfall_df['February'].iloc[1:62].max()
mch1 = monthly_rainfall_df['March'].iloc[1:62].max()
apr1 = monthly_rainfall_df['April'].iloc[1:62].max()
may1 = monthly_rainfall_df['May'].iloc[1:62].max()
jun1 = monthly_rainfall_df['June'].iloc[1:62].max()
jul1 = monthly_rainfall_df['July'].iloc[0:61].max()
aug1 = monthly_rainfall_df['August'].iloc[0:61].max()
sep1 = monthly_rainfall_df['September'].iloc[0:61].max()
octb1 = monthly_rainfall_df['October'].iloc[0:61].max()
nov1 = monthly_rainfall_df['November'].iloc[0:61].max()
dec1 = monthly_rainfall_df['December'].iloc[0:61].max()

In [69]:
max_one_day = [jan1, feb1, mch1, apr1, may1, jun1, jul1, aug1, sep1, octb1, nov1, dec1]

In [70]:
monthly_stats_df.insert(2, 'Max Monthly Rainfall', max_one_day)

### 3.3.5 Insert 'Average' Column

In [71]:
jan2 = ((monthly_rainfall_df['January'].iloc[1:62].sum())/monthly_len)
feb2 = ((monthly_rainfall_df['February'].iloc[1:62].sum())/monthly_len)
mch2 = ((monthly_rainfall_df['March'].iloc[1:62].sum())/monthly_len)
apr2 = ((monthly_rainfall_df['April'].iloc[1:62].sum())/monthly_len)
may2 = ((monthly_rainfall_df['May'].iloc[1:62].sum())/monthly_len)
jun2 = ((monthly_rainfall_df['June'].iloc[1:62].sum())/monthly_len)
jul2 = ((monthly_rainfall_df['July'].iloc[0:61].sum())/monthly_len)
aug2 = ((monthly_rainfall_df['August'].iloc[0:61].sum())/monthly_len)
sep2 = ((monthly_rainfall_df['September'].iloc[0:61].sum())/monthly_len)
octb2 = ((monthly_rainfall_df['October'].iloc[0:61].sum())/monthly_len)
nov2 = ((monthly_rainfall_df['November'].iloc[0:61].sum())/monthly_len)
dec2 = ((monthly_rainfall_df['December'].iloc[0:61].sum())/monthly_len)

In [72]:
average = [jan2, feb2, mch2, apr2, may2, jun2, jul2, aug2, sep2, octb2, nov2, dec2]

In [73]:
monthly_stats_df.insert(3, 'Average', average)

### 3.3.6 Insert 'Standard Deviation' Column

In [74]:
jan3 = (statistics.stdev(monthly_rainfall_df['January'].iloc[1:62]))
feb3 = (statistics.stdev(monthly_rainfall_df['February'].iloc[1:62]))
mch3 = (statistics.stdev(monthly_rainfall_df['March'].iloc[1:62]))
apr3 = (statistics.stdev(monthly_rainfall_df['April'].iloc[1:62]))
may3 = (statistics.stdev(monthly_rainfall_df['May'].iloc[1:62]))
jun3 = (statistics.stdev(monthly_rainfall_df['June'].iloc[1:62]))
jul3 = (statistics.stdev(monthly_rainfall_df['July'].iloc[0:61]))
aug3 = (statistics.stdev(monthly_rainfall_df['August'].iloc[0:61]))
sep3 = (statistics.stdev(monthly_rainfall_df['September'].iloc[0:61]))
octb3 = (statistics.stdev(monthly_rainfall_df['October'].iloc[0:61]))
nov3 = (statistics.stdev(monthly_rainfall_df['November'].iloc[0:61]))
dec3 = (statistics.stdev(monthly_rainfall_df['December'].iloc[0:61]))

In [75]:
stdep = [jan3, feb3, mch3, apr3, may3, jun3, jul3, aug3, sep3, octb3, nov3, dec3]

In [76]:
monthly_stats_df.insert(4, 'Standard Deviation', stdep)

### 3.3.7 Insert 'Mean' Column

In [77]:
jan4 = (statistics.mean(monthly_rainfall_df['January'].iloc[1:62]))
feb4 = (statistics.mean(monthly_rainfall_df['February'].iloc[1:62]))
mch4 = (statistics.mean(monthly_rainfall_df['March'].iloc[1:62]))
apr4 = (statistics.mean(monthly_rainfall_df['April'].iloc[1:62]))
may4 = (statistics.mean(monthly_rainfall_df['May'].iloc[1:62]))
jun4 = (statistics.mean(monthly_rainfall_df['June'].iloc[1:62]))
jul4 = (statistics.mean(monthly_rainfall_df['July'].iloc[0:61]))
aug4 = (statistics.mean(monthly_rainfall_df['August'].iloc[0:61]))
sep4 = (statistics.mean(monthly_rainfall_df['September'].iloc[0:61]))
octb4 = (statistics.mean(monthly_rainfall_df['October'].iloc[0:61]))
nov4 = (statistics.mean(monthly_rainfall_df['November'].iloc[0:61]))
dec4 = (statistics.mean(monthly_rainfall_df['December'].iloc[0:61]))

In [78]:
mean = [jan4, feb4, mch4, apr4, may4, jun4, jul4, aug4, sep4, octb4, nov4, dec4]

In [79]:
monthly_stats_df.insert(5, 'Mean', mean)

### 3.3.8 Display 'monthly_stats_df' Dataframe

In [80]:
monthly_stats_df

,Month,Min Monthly Rainfall,Max Monthly Rainfall,Average,Standard Deviation,Mean
0,January,4.1,389.8,69.146774,63.483843,70.280328
1,February,8.6,291.6,70.843548,61.864496,72.004918
2,March,7.6,261.4,77.190323,52.615988,78.455738
3,April,12.7,221.0,85.251613,44.243804,86.649180
4,May,23.0,307.8,107.630645,51.977670,109.395082
5,June,47.0,209.0,113.974194,37.273648,115.842623
6,July,15.4,308.4,128.287097,53.141057,130.390164
7,August,36.6,221.1,112.727419,45.592356,114.575410
8,September,38.0,210.6,93.866129,38.840496,95.404918
9,October,11.0,182.7,83.019355,39.217164,84.380328


**************************************************************************************************

## 3.4 Create 'seasonal_rainfall_df' Dataframe

### 3.4.1 Create 'seasonal_rainfall_df' Dataframe

In [81]:
seasonal_rainfall_df = pd.DataFrame()

### 3.4.2 Change ' ' values to 0, to do Calculations

In [82]:
january = monthly_rainfall_df['January'].replace('', 0)
february = monthly_rainfall_df['February'].replace('', 0)
march = monthly_rainfall_df['March'].replace('', 0)
april = monthly_rainfall_df['April'].replace('', 0)
may = monthly_rainfall_df['May'].replace('', 0)
june = monthly_rainfall_df['June'].replace('', 0)
july = monthly_rainfall_df['July'].replace('', 0)
august = monthly_rainfall_df['August'].replace('', 0)
september = monthly_rainfall_df['September'].replace('', 0)
october = monthly_rainfall_df['October'].replace('', 0)
november = monthly_rainfall_df['November'].replace('', 0)
december = monthly_rainfall_df['December'].replace('', 0)

### 3.4.3 Insert 'Year' Column

In [83]:
seasonal_rainfall_df.insert(0, 'Year', year_list)

### 3.4.4 Insert 'Summer' Column

###### Summer: December, January & February

In [84]:
summer = []
for i in range(len(seasonal_rainfall_df)):
    summer.append(december[i] + january[i] + february[i])

In [85]:
seasonal_rainfall_df.insert(1, 'Summer (Dec - Feb)', summer)

### 3.4.5 Insert 'Autumn' Column

###### Autumn: March, April & May

In [86]:
autumn = []
for i in range(len(seasonal_rainfall_df)):
    autumn.append(march[i] + april[i] + may[i])

In [87]:
seasonal_rainfall_df.insert(2, 'Autumn (March - May)', autumn)

### 3.4.6 Insert 'Winter' Column

###### Winter: June, July & August

In [88]:
winter = []
for i in range(len(seasonal_rainfall_df)):
    winter.append(june[i] + july[i] + august[i])

In [89]:
seasonal_rainfall_df.insert(3, 'Winter (Jun - Aug)', winter)

### 3.4.7 Insert 'Spring' Column

###### Spring: September, October & November

In [90]:
spring = []
for i in range(len(seasonal_rainfall_df)):
    spring.append(september[i] + october[i] + november[i])

In [91]:
seasonal_rainfall_df.insert(4, 'Spring (Sep - Nov)', spring)

### 3.4.8 Display 'seasonal_rainfall_df' Dataframe

In [92]:
#No data during Jan - June 1962 and July - December 2023. For accurate calculations, change Summer & Autmn 1962 and Winter & Spring values.
seasonal_rainfall_df['Summer (Dec - Feb)'].iloc[0] = ''
seasonal_rainfall_df['Autumn (March - May)'].iloc[0] = ''
seasonal_rainfall_df['Winter (Jun - Aug)'].iloc[61] = ''
seasonal_rainfall_df['Spring (Sep - Nov)'].iloc[61] = ''

In [93]:
seasonal_rainfall_df

,Year,Summer (Dec - Feb),Autumn (March - May),Winter (Jun - Aug),Spring (Sep - Nov)
0,1962,,,267.5,399.5
1,1963,231.2,247.0,314.8,190.4
2,1964,224.3,254.3,450.0,304.3
3,1965,362.3,285.1,502.3,201.9
4,1966,343.1,346.7,329.4,276.7
...,...,...,...,...,...
57,2019,103.2,165.6,438.8,234.4
58,2020,66.2,170.6,381.6,175.4
59,2021,196.0,177.6,284.0,254.6
60,2022,186.2,216.6,573.0,378.0


**************************************************************************************************

## 3.5 Create 'seasonal_stats_df' Dataframe

### 3.5.1 Create 'seasonal_stats_df' Dataframe

In [94]:
seasonal_stats_df = pd.DataFrame()

### 3.5.2 Insert 'Seasons' Column

In [95]:
seasons_index = ['Summer (Dec - Feb)', 'Autumn (March - May)', 'Winter (Jun - Aug)', 'Spring (Sep - Nov)']

In [96]:
seasonal_stats_df.insert(0, 'Seasons', seasons_index)

### 3.5.3 Insert 'Min Seasonal Rainfall' Column

In [97]:
sum1 = seasonal_rainfall_df['Summer (Dec - Feb)'].iloc[1:62].min()
aut1 = seasonal_rainfall_df['Autumn (March - May)'].iloc[1:62].min()
win1 = seasonal_rainfall_df['Winter (Jun - Aug)'].iloc[0:61].min()
spr1 = seasonal_rainfall_df['Spring (Sep - Nov)'].iloc[0:61].min()

In [98]:
min_season = [sum1, aut1, win1, spr1]

In [99]:
seasonal_stats_df.insert(1, 'Min Seasonal Rainfall', min_season)

### 3.5.4 Insert 'Max Seasonal Rainfall' Column

In [100]:
sum2 = seasonal_rainfall_df['Summer (Dec - Feb)'].iloc[1:62].max()
aut2 = seasonal_rainfall_df['Autumn (March - May)'].iloc[1:62].max()
win2 = seasonal_rainfall_df['Winter (Jun - Aug)'].iloc[0:61].max()
spr2 = seasonal_rainfall_df['Spring (Sep - Nov)'].iloc[0:61].max()

In [101]:
max_season = [sum2, aut2, win2, spr2]

In [102]:
seasonal_stats_df.insert(2, 'Max Seasonal Rainfall', max_season)

### 3.5.5 Insert 'Average' Column

In [103]:
sum3 = ((seasonal_rainfall_df['Summer (Dec - Feb)'].iloc[1:62].sum())/monthly_len)
aut3 = ((seasonal_rainfall_df['Autumn (March - May)'].iloc[1:62].sum())/monthly_len)
win3 = ((seasonal_rainfall_df['Winter (Jun - Aug)'].iloc[0:61].sum())/monthly_len)
spr3 = ((seasonal_rainfall_df['Spring (Sep - Nov)'].iloc[0:61].sum())/monthly_len)

In [104]:
average_season = [sum3, aut3, win3, spr3]

In [105]:
seasonal_stats_df.insert(3, 'Average', average_season)

### 3.5.6 Insert 'Standard Deviation' Column

In [106]:
sum4 = (statistics.stdev(seasonal_rainfall_df['Summer (Dec - Feb)'].iloc[1:62]))
aut4 = (statistics.stdev(seasonal_rainfall_df['Autumn (March - May)'].iloc[1:62]))
win4 = (statistics.stdev(seasonal_rainfall_df['Winter (Jun - Aug)'].iloc[0:61]))
spr4 = (statistics.stdev(seasonal_rainfall_df['Spring (Sep - Nov)'].iloc[0:61]))

In [107]:
std_season = [sum4, aut4, win4, spr4]

In [108]:
seasonal_stats_df.insert(4, 'Standard Deviation', std_season)

### 3.5.7 Insert 'Mean' Column

In [109]:
sum5 = (statistics.mean(seasonal_rainfall_df['Summer (Dec - Feb)'].iloc[1:62]))
aut5 = (statistics.mean(seasonal_rainfall_df['Autumn (March - May)'].iloc[1:62]))
win5 = (statistics.mean(seasonal_rainfall_df['Winter (Jun - Aug)'].iloc[0:61]))
spr5 = (statistics.mean(seasonal_rainfall_df['Spring (Sep - Nov)'].iloc[0:61]))

In [110]:
mean_season = [sum5, aut5, win5, spr5]

In [111]:
seasonal_stats_df.insert(5, 'Mean', mean_season)

### 3.5.8 Display 'seasonal_stats_df' Dataframe

In [112]:
seasonal_stats_df

,Seasons,Min Seasonal Rainfall,Max Seasonal Rainfall,Average,Standard Deviation,Mean
0,Summer (Dec - Feb),66.2,625.6,222.504839,116.681967,226.152459
1,Autumn (March - May),165.6,547.0,270.072581,79.182214,274.500000
2,Winter (Jun - Aug),194.2,573.0,353.675806,87.040825,359.473770
3,Spring (Sep - Nov),122.7,399.5,251.070968,66.261375,255.186885


**************************************************************************************************
**************************************************************************************************

# 4. Data Visualisation: Impact of La Niña on Auckland, New Zealand January 2023

## 4.1 Create 'gumbel_est_calculations_df' Dataframe

### 4.1.1 Create 'gumbel_est_calculations_df' Dataframe

In [113]:
gumbel_est_calculations_df = pd.DataFrame()

### 4.1.2 Insert 'Year' Column

In [114]:
gumbel_est_calculations_df.insert(0, 'Year', year_list)

### 4.1.3 Insert 'Max One-Day Rainfall' Column

#### 4.1.3.1 Get Maximum One Day Rainfall (mm)

###### Create 'year_rainfall_df' Dataframe

In [115]:
year_rainfall_df = pd.DataFrame()

###### Insert 'YYYY' Column

In [116]:
year_date = []
for i in range(len(rainfall_data_df)):
    yyyy_date = str(rainfall_data_df['Date'].iloc[i])[:4]
    year_date.append(int(yyyy_date))

year_rainfall_df.insert(0, 'YYYY', year_date)

###### Insert 'Rainfall (mm/d)' Column

In [117]:
year_rainfall_df.insert(1, 'Rainfall (mm/d)', rainfall_data_df['Rainfall (mm/d)'])

In [118]:
max_1day_rain = year_rainfall_df.groupby('YYYY')['Rainfall (mm/d)'].max()

#### 4.1.3.2 Insert 'Max One-Day Rainfall' Column

In [119]:
max_one_day = []
for i in range(len(max_1day_rain)):
    max_one_day.append(max_1day_rain.iloc[i])

In [120]:
gumbel_est_calculations_df.insert(1, 'Max One-Day Rainfall', max_one_day)

### 4.1.4 Insert 'Observations (N)' Column

In [121]:
observation_n = len(gumbel_est_calculations_df)

In [122]:
n_obs = []
for i in range(len(gumbel_est_calculations_df)):
    n_obs.append(observation_n)

In [123]:
gumbel_est_calculations_df.insert(2, 'Observations (N)', n_obs)

### 4.1.5 Insert 'Rank (i)' Column

In [124]:
rank_i = []
for i in range(len(gumbel_est_calculations_df)):
    rank_i.append(i+1)

In [125]:
gumbel_est_calculations_df = gumbel_est_calculations_df.sort_values(['Max One-Day Rainfall'], ascending = [False])

In [126]:
gumbel_est_calculations_df.insert(3, 'Rank (i)', rank_i)

### 4.1.6 Insert 'Exceedance (P)' Column

###### P=i/(N+1)

In [127]:
p_exceed = []
for i in range(len(gumbel_est_calculations_df)):
    p_exceed.append((gumbel_est_calculations_df['Rank (i)'].iloc[i])/((gumbel_est_calculations_df['Observations (N)'].iloc[i])+1))

In [128]:
gumbel_est_calculations_df.insert(4, 'Exceedance (P)', p_exceed)

### 4.1.7 Insert 'Non-Exceedance (Q)' Column

###### Q = 1 - P

In [129]:
q_non = []
for i in range(len(gumbel_est_calculations_df)):
    q_non.append(1 - (gumbel_est_calculations_df['Exceedance (P)'].iloc[i]))

In [130]:
gumbel_est_calculations_df.insert(5, 'Non-Exceedance (Q)', q_non)

### 4.1.8 Insert 'Gumbel Est. Return Period (T_a)' Column

###### T_a = 1/P

In [131]:
t_a = []
for i in range(len(gumbel_est_calculations_df)):
    t_a.append(1 / (gumbel_est_calculations_df['Exceedance (P)'].iloc[i]))

In [132]:
gumbel_est_calculations_df.insert(6, 'Gumbel Est. Return Period (T_a)', t_a)

### 4.1.9 Insert 'Reduced Variable (Y)' Column

###### -ln(-ln(Q))

In [133]:
y_redvar = []
for i in range(len(gumbel_est_calculations_df)):
    y_redvar.append(-np.log(-np.log(gumbel_est_calculations_df['Non-Exceedance (Q)'].iloc[i])))

In [134]:
gumbel_est_calculations_df.insert(7, 'Reduced Variable (Y)', y_redvar)

### 4.1.10 Insert 'Gumble Estimate' Column

###### (Sigma*[Y (Reduced Var)]) + mu

#### 5.1.10.1 Define Variables

######  s_y = Std Dev of Reduced Var (Y)

In [135]:
s_y = np.std(y_redvar)

###### s_R = Std Dev of Max One-Day Rainfall

In [136]:
s_R = np.std(gumbel_est_calculations_df['Max One-Day Rainfall'])

###### R_max_gem = Average of Max One-Day Rainfall

In [137]:
R_max_gem = np.average(gumbel_est_calculations_df['Max One-Day Rainfall'])

###### y_gem = Average of Reduced Var (Y)

In [138]:
y_gem = np.average(y_redvar)

###### s_y = Std Dev of Reduced Var (Y)

In [139]:
s_y = np.std(y_redvar)

###### Sigma = s_R / s_y

In [140]:
sigma = s_R / s_y

###### mu = R_max_gem-((s_R)*(y_gem/s_y))

In [141]:
mu = R_max_gem-((s_R)*(y_gem / s_y))

#### 4.1.10.2 (Sigma*[Y (Reduced Var)]) + mu

In [142]:
gumbel_est = []
for i in range(len(gumbel_est_calculations_df)):
    gumbel_est.append(sigma*(y_redvar[i])+mu)

In [143]:
gumbel_est_calculations_df.insert(8, 'Gumbel Estimate', gumbel_est)

### 4.1.11 Display 'gumbel_est_calculations_df' Dataframe

###### Sort Dataframe by Year (Ascending)

In [144]:
gumbel_est_calculations_df = gumbel_est_calculations_df.sort_values(by='Year', ascending = True) 

###### Display 'gumbel_est_calculations_df' Dataframe

In [145]:
gumbel_est_calculations_df

,Year,Max One-Day Rainfall,Observations (N),Rank (i),Exceedance (P),Non-Exceedance (Q),Gumbel Est. Return Period (T_a),Reduced Variable (Y),Gumbel Estimate
0,1962,51.3,62,46,0.730159,0.269841,1.369565,-0.269967,44.330619
1,1963,56.4,62,40,0.634921,0.365079,1.575000,-0.007611,51.577249
2,1964,44.7,62,52,0.825397,0.174603,1.211538,-0.556892,36.405358
3,1965,70.4,62,16,0.253968,0.746032,3.937500,1.227627,85.696256
4,1966,66.5,62,18,0.285714,0.714286,3.500000,1.089240,81.873814
...,...,...,...,...,...,...,...,...,...
57,2019,41.4,62,55,0.873016,0.126984,1.145455,-0.724497,31.775861
58,2020,65.6,62,21,0.333333,0.666667,3.000000,0.902720,76.721893
59,2021,42.2,62,54,0.857143,0.142857,1.166667,-0.665730,33.399099
60,2022,62.8,62,30,0.476190,0.523810,2.100000,0.435985,63.830017


**************************************************************************************************

## 4.2 Create 'gumbel_variables_df' Dataframe

### 4.2.1 Create 'gumbel_variables_df' Dataframe

In [146]:
gumbel_variables_df = pd.DataFrame()

### 4.2.2 Insert 'Variable' Column

In [147]:
variable = ['s_y', 's_R', 'R_max_gem', 'y_gem', 's_y', 'Sigma', 'mu']

In [148]:
gumbel_variables_df.insert(0, 'Variable', variable)

### 4.2.3 Insert 'Equation' Column

In [149]:
equation = ['s_y = Std Dev of Y (Reduced Var)', 's_R = Std Dev of Max 1-Day Rainfall (mm)', 
            'R_max_gem = Average of Max 1-Day Rainfall (mm)', 'y_gem = Average of Y (Reduced Var)', 
            's_y = Std Dev of Y (Reduced Var)', 'Sigma = s_R / s_y', 
            'mu = R_max_gem-((s_R)*(y_gem/s_y))']

In [150]:
gumbel_variables_df.insert(1, 'Equation', equation)

### 4.2.4 Insert 'Value' Column

In [151]:
value = [s_y, s_R, R_max_gem, y_gem, s_y, sigma, mu]

In [152]:
gumbel_variables_df.insert(2, 'Value', value)

### 4.2.5 Display 'gumbel_variables_df' Dataframe

In [153]:
gumbel_variables_df

,Variable,Equation,Value
0,s_y,s_y = Std Dev of Y (Reduced Var),1.177024
1,s_R,s_R = Std Dev of Max 1-Day Rainfall (mm),32.511049
2,R_max_gem,R_max_gem = Average of Max 1-Day Rainfall (mm),67.053226
3,y_gem,y_gem = Average of Y (Reduced Var),0.552678
4,s_y,s_y = Std Dev of Y (Reduced Var),1.177024
5,Sigma,Sigma = s_R / s_y,27.621401
6,mu,mu = R_max_gem-((s_R)*(y_gem/s_y)),51.787489


**************************************************************************************************

## 4.3 Create 'gumbel_est_df' Dataframe

### 4.3.1 Create 'gumbel_est_df' Dataframe

In [154]:
gumbel_est_df = pd.DataFrame()

### 4.3.2 Insert 'Return Period (Year)' Column

In [155]:
return_period = [1, 1.5, 2, 5, 10, 20, 35, 50, 60, 100, 200, 500, 1000]

In [156]:
gumbel_est_df.insert(0, 'Return Period (Year)', return_period)

### 4.3.3 Insert 'Gumbel Est. Return Period (T_a)' Column

###### (1)/(1-e^(-1/['Return Period T (Year)']))

In [157]:
t_a_gumbel = []
for i in range(len(gumbel_est_df)):
    t_a_gumbel.append(1 / (1 - (math.exp(float(-1 / (gumbel_est_df['Return Period (Year)'].iloc[i]))))))

In [158]:
gumbel_est_df.insert(1, 'Gumbel Est. Return Period (T_a)', t_a_gumbel)

### 4.3.4 Insert 'Reduced Variable (Y)' Column

###### -ln(-ln(1-(1/['Gumbel Est. Return Period (T_a)'])))

In [159]:
y_gumbel = []
for i in range(len(gumbel_est_df)):
    y_gumbel.append(-np.log(-np.log(1 - (1 / (gumbel_est_df['Gumbel Est. Return Period (T_a)'].iloc[i])))))

In [160]:
gumbel_est_df.insert(2, 'Reduced Variable (Y)', y_gumbel)

### 4.3.5 Insert 'Gumbel Estimate' Column

###### sigma* y_gumbel[] + mu

In [161]:
est_gumbel = []
for i in range(len(gumbel_est_df)):
    est_gumbel.append(sigma * (gumbel_est_df['Reduced Variable (Y)'].iloc[i]) + mu)

In [162]:
gumbel_est_df.insert(3, 'Gumbel Estimate', est_gumbel)

### 4.3.6 Display 'gumbel_est_df' Dataframe

In [163]:
gumbel_est_df

,Return Period (Year),Gumbel Est. Return Period (T_a),Reduced Variable (Y),Gumbel Estimate
0,1.0,1.581977,-0.000000,51.787489
1,1.5,2.055148,0.405465,62.987003
2,2.0,2.541494,0.693147,70.933185
3,5.0,5.516656,1.609438,96.242419
4,10.0,10.508332,2.302585,115.388116
5,20.0,20.504166,2.995732,134.533812
6,35.0,35.502381,3.555348,149.991185
7,50.0,50.501667,3.912023,159.843046
8,60.0,60.501389,4.094345,164.879023
9,100.0,100.500833,4.605170,178.988743


**************************************************************************************************

## 4.4 Create 'january_2023_stats_df' Dataframe

### 4.4.1 Create 'january_2023_stats_df' Dataframe

In [164]:
january_2023_stats_df = pd.DataFrame()

### 4.4.2 Insert 'Statistic Description' Column

In [165]:
stat_description = ['Min One-Day Rainfall', 'Max One-Day Rainfall', 
                    'Average One-Day Rainfall', 'Mean', 'Standard Deviation']

In [166]:
january_2023_stats_df.insert(0, 'Statistic Description', stat_description)

### 4.4.3 Create 'jan_2023_daily_df' and 'jan_avg_daily_df' Dataframe

#### 4.4.3.1 Create 'jan_stats_calc_df' Dataframe

In [167]:
jan_stats_calc_df = rainfall_data_df
jan_stats_calc_df = jan_stats_calc_df.drop(columns=['Station'])

In [168]:
for i in range(len(rainfall_data_df)):
    if str(rainfall_data_df['Date'].iloc[i])[5:7] != '01':
        jan_stats_calc_df = jan_stats_calc_df.drop([i])

In [169]:
index_list = []
for i in range(len(jan_stats_calc_df)):
    index_list.append(i)
jan_stats_calc_df.insert(0, '', index_list)

jan_stats_calc_df = jan_stats_calc_df.set_index('')

#### 4.4.3.2 Create 'jan_2023_daily_df' Dataframe

In [170]:
jan_2023_daily_df = jan_stats_calc_df.iloc[1836:]

index_list = []
for i in range(len(jan_2023_daily_df)):
    index_list.append(i)
jan_2023_daily_df.insert(0, '', index_list)

jan_2023_daily_df = jan_2023_daily_df.set_index('')

#### 4.4.3.3 Create 'jan_avg_daily_df' Dataframe

In [171]:
jan_avg_daily_df = jan_stats_calc_df.iloc[:1836]

### 4.4.4 Insert 'January (1963 - 2022)' Column

#### 4.4.4.1 Define Variables

In [172]:
min_1_day_avg = jan_avg_daily_df['Rainfall (mm/d)'].min()

In [173]:
max_1_day_avg = jan_avg_daily_df['Rainfall (mm/d)'].max()

In [174]:
avg_rainfall = (jan_avg_daily_df['Rainfall (mm/d)'].sum())/(len(jan_avg_daily_df['Rainfall (mm/d)']))

In [175]:
mean_rainfall = statistics.mean(jan_avg_daily_df['Rainfall (mm/d)'])

In [176]:
std_rainfall = statistics.stdev(jan_avg_daily_df['Rainfall (mm/d)'])

#### 4.4.4.2 Add Variable to List

In [177]:
jan_1963_2022 = [min_1_day_avg, max_1_day_avg, avg_rainfall, mean_rainfall, std_rainfall]

#### 4.4.4.3 Insert 'January Avg (1963 - 2022)' Column

In [178]:
january_2023_stats_df.insert(1, 'January (1963 - 2022)', jan_1963_2022)

### 4.4.5 Insert 'January 2023' Column

#### 4.4.5.1 Define Variables

In [179]:
min_1_day_2023 = jan_2023_daily_df['Rainfall (mm/d)'].min()

In [180]:
max_1_day_2023 = jan_2023_daily_df['Rainfall (mm/d)'].max()

In [181]:
avg_2023 = (jan_2023_daily_df['Rainfall (mm/d)'].sum())/(len(jan_2023_daily_df['Rainfall (mm/d)']))

In [182]:
mean_2023 = statistics.mean(jan_2023_daily_df['Rainfall (mm/d)'])

In [183]:
std_2023 = statistics.stdev(jan_2023_daily_df['Rainfall (mm/d)'])

#### 4.4.5.2 Add Variable to List

In [184]:
jan_2023 = [min_1_day_2023, max_1_day_2023, avg_2023, mean_2023, std_2023]

#### 4.4.5.3 Insert 'January 2023' Column

In [185]:
january_2023_stats_df.insert(2, 'January 2023', jan_2023)

### 4.4.6 Display 'january_2023_stats_df' Dataframe

In [186]:
january_2023_stats_df

,Statistic Description,January (1963 - 2022),January 2023
0,Min One-Day Rainfall,0.000000,0.000000
1,Max One-Day Rainfall,75.700000,245.000000
2,Average One-Day Rainfall,2.122712,12.574194
3,Mean,2.122712,12.574194
4,Standard Deviation,6.839043,43.750771


**************************************************************************************************
**************************************************************************************************

# 5. Excel Modification

## 5.1 Save all the DataFrames to an Excel Spreadsheet

### 5.1.1 Create Empty Dataframe

In [187]:
empty_df = pd.DataFrame()

### 5.1.2 Define Dataframes

In [188]:
dataframe1 = rainfall_data_df
dataframe2 = monthly_rainfall_df
dataframe3 = monthly_stats_df
dataframe4 = seasonal_rainfall_df
dataframe5 = seasonal_stats_df
dataframe9 = gumbel_est_calculations_df
dataframe10 = january_2023_stats_df

### 5.1.3 Create Excel Spreadsheet

In [189]:
with pd.ExcelWriter('output_file.xlsx', engine='xlsxwriter') as writer:
    dataframe1.to_excel(writer, sheet_name='Auckland Aero Rainfall Data', index=False)
    dataframe2.to_excel(writer, sheet_name='Monthly Rainfall', index=False)
    dataframe3.to_excel(writer, sheet_name='Monthly Statistics', index=False)
    dataframe4.to_excel(writer, sheet_name='Seasonal Rainfall', index=False)
    dataframe5.to_excel(writer, sheet_name='Seasonal Statistics', index=False)
    dataframe9.to_excel(writer, sheet_name='Gumbel Estimate', index=False)
    dataframe10.to_excel(writer, sheet_name='Auckland Jan 2023 Stats', index=False)

### 5.1.4 Set Excel Spreadsheet Zoom to 60%

In [190]:
def is_desired_worksheet(ws):
    desired_names = ['Auckland Aero Rainfall Data', 'Monthly Rainfall', 'Monthly Statistics',
                     'Seasonal Rainfall', 'Seasonal Statistics', 'Gumbel Estimate', 
                     'Auckland Jan 2023 Stats']
    return ws.title in desired_names

In [191]:
wb = load_workbook('output_file.xlsx')
for ws in wb.worksheets:
    if is_desired_worksheet(ws):
        ws.sheet_view.zoomScale = 60
wb.save('output_file.xlsx')
wb.close()

### 5.1.5 Remove Gridlines

In [192]:
wb = load_workbook('output_file.xlsx')

for ws in wb.worksheets:
    ws.sheet_view.showGridLines = False

for ws in wb.worksheets:
    ws.sheet_view.showRowColHeaders = False

wb.save('output_file.xlsx')
wb.close()

**************************************************************************************************

## 5.2 Export Tables

###### gumbel_variables_df

In [193]:
table10_1 = gumbel_variables_df
dfi.export(table10_1,"gumbel_variables.png")

###### gumbel_est_df

In [194]:
table10_2 = gumbel_est_df
dfi.export(table10_2,"gumbel_est.png")

In [195]:
time.sleep(2)

**************************************************************************************************

## 5.3 Modify Excel Spreadsheet

### 5.3.1 Change Data Types

In [196]:
wb = load_workbook('output_file.xlsx')
ws = wb['Auckland Aero Rainfall Data']

date_change = []
for i in range(len(rainfall_data_df)+2):
    if i != 0:
        date_change.append('B' + str(i))
        
for i in range(len(date_change)):
        ws[(date_change[i])].number_format = 'YYYY-MM-DD'

wb.save('output_file.xlsx')
wb.close()

### 5.3.2 Change Row width to Fit Contents

In [197]:
wb = load_workbook('output_file.xlsx')
ws = wb['Auckland Aero Rainfall Data']
ws.column_dimensions['A'].width = 17
ws.column_dimensions['B'].width = 11.5
ws.column_dimensions['C'].width = 18.5
wb.save('output_file.xlsx')
wb.close()

In [198]:
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Rainfall']
ws.column_dimensions['A'].width = 10
ws.column_dimensions['B'].width = 12
ws.column_dimensions['C'].width = 12
ws.column_dimensions['D'].width = 12
ws.column_dimensions['E'].width = 12
ws.column_dimensions['F'].width = 12
ws.column_dimensions['G'].width = 12
ws.column_dimensions['H'].width = 12
ws.column_dimensions['I'].width = 12
ws.column_dimensions['J'].width = 12
ws.column_dimensions['K'].width = 12
ws.column_dimensions['L'].width = 12
ws.column_dimensions['M'].width = 12
ws.column_dimensions['N'].width = 18
wb.save('output_file.xlsx')
wb.close()

In [199]:
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Statistics']
ws.column_dimensions['A'].width = 12
ws.column_dimensions['B'].width = 27
ws.column_dimensions['C'].width = 27
ws.column_dimensions['D'].width = 12
ws.column_dimensions['E'].width = 24
ws.column_dimensions['F'].width = 12
wb.save('output_file.xlsx')
wb.close()

In [200]:
wb = load_workbook('output_file.xlsx')
ws = wb['Seasonal Rainfall']
ws.column_dimensions['A'].width = 10
ws.column_dimensions['B'].width = 27
ws.column_dimensions['C'].width = 27
ws.column_dimensions['D'].width = 27
ws.column_dimensions['E'].width = 27
wb.save('output_file.xlsx')
wb.close()

In [201]:
wb = load_workbook('output_file.xlsx')
ws = wb['Seasonal Statistics']
ws.column_dimensions['A'].width = 27
ws.column_dimensions['B'].width = 27
ws.column_dimensions['C'].width = 27
ws.column_dimensions['D'].width = 15
ws.column_dimensions['E'].width = 27
ws.column_dimensions['F'].width = 15
wb.save('output_file.xlsx')
wb.close()

In [202]:
wb = load_workbook('output_file.xlsx')
ws = wb['Gumbel Estimate']
ws.column_dimensions['A'].width = 10
ws.column_dimensions['B'].width = 30
ws.column_dimensions['C'].width = 30
ws.column_dimensions['D'].width = 20
ws.column_dimensions['E'].width = 30
ws.column_dimensions['F'].width = 30
ws.column_dimensions['G'].width = 39
ws.column_dimensions['H'].width = 30
ws.column_dimensions['I'].width = 30
wb.save('output_file.xlsx')
wb.close()

In [203]:
wb = load_workbook('output_file.xlsx')
ws = wb['Auckland Jan 2023 Stats']
ws.column_dimensions['A'].width = 27
ws.column_dimensions['B'].width = 27
ws.column_dimensions['C'].width = 27
wb.save('output_file.xlsx')
wb.close()

### 5.3.3 Change Top Row Colour to Dark Orange (#E76400)

In [204]:
wb = load_workbook('output_file.xlsx')
ws = wb['Auckland Aero Rainfall Data']
ws['A1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['B1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['C1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [205]:
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Rainfall']
ws['A1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['B1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['C1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['D1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['E1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['F1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['G1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['H1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['I1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['J1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['K1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['L1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['M1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['N1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [206]:
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Statistics']
ws['A1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['B1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['C1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['D1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['E1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['F1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [207]:
wb = load_workbook('output_file.xlsx')
ws = wb['Seasonal Rainfall']
ws['A1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['B1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['C1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['D1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['E1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [208]:
wb = load_workbook('output_file.xlsx')
ws = wb['Seasonal Statistics']
ws['A1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['B1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['C1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['D1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['E1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['F1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [209]:
wb = load_workbook('output_file.xlsx')
ws = wb['Gumbel Estimate']
ws['A1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['B1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['C1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['D1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['E1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['F1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['G1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['H1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['I1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [210]:
wb = load_workbook('output_file.xlsx')
ws = wb['Auckland Jan 2023 Stats']
ws['A1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['B1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
ws['C1'].fill = PatternFill(start_color = 'E76400', end_color = 'E76400', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

### 5.3.3 Change every second Row Colour to Light Orange (#FCCA88)

###### Auckland Aero Rainfall Data

In [211]:
colourA = []
for i in range(len(rainfall_data_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourA.append('A' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Auckland Aero Rainfall Data']
for i in range(len(colourA)):
    ws[(colourA[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [212]:
colourB = []
for i in range(len(rainfall_data_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourB.append('B' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Auckland Aero Rainfall Data']
for i in range(len(colourB)):
    ws[(colourB[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [213]:
colourC = []
for i in range(len(rainfall_data_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourC.append('C' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Auckland Aero Rainfall Data']
for i in range(len(colourC)):
    ws[(colourC[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

###### Monthly Rainfall

In [214]:
colourD = []
for i in range(len(monthly_rainfall_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourD.append('A' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Rainfall']
for i in range(len(colourD)):
    ws[(colourD[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [215]:
colourE = []
for i in range(len(monthly_rainfall_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourE.append('B' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Rainfall']
for i in range(len(colourE)):
    ws[(colourE[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [216]:
colourF = []
for i in range(len(monthly_rainfall_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourF.append('C' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Rainfall']
for i in range(len(colourF)):
    ws[(colourF[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [217]:
colourG = []
for i in range(len(monthly_rainfall_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourG.append('D' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Rainfall']
for i in range(len(colourG)):
    ws[(colourG[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [218]:
colourH = []
for i in range(len(monthly_rainfall_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourH.append('E' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Rainfall']
for i in range(len(colourH)):
    ws[(colourH[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [219]:
colourI = []
for i in range(len(monthly_rainfall_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourI.append('F' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Rainfall']
for i in range(len(colourI)):
    ws[(colourI[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [220]:
colourJ = []
for i in range(len(monthly_rainfall_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourJ.append('G' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Rainfall']
for i in range(len(colourJ)):
    ws[(colourJ[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [221]:
colourK = []
for i in range(len(monthly_rainfall_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourK.append('H' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Rainfall']
for i in range(len(colourK)):
    ws[(colourK[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [222]:
colourL = []
for i in range(len(monthly_rainfall_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourL.append('I' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Rainfall']
for i in range(len(colourL)):
    ws[(colourL[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [223]:
colourM = []
for i in range(len(monthly_rainfall_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourM.append('J' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Rainfall']
for i in range(len(colourM)):
    ws[(colourM[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [224]:
colourN = []
for i in range(len(monthly_rainfall_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourN.append('K' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Rainfall']
for i in range(len(colourN)):
    ws[(colourN[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [225]:
colourO = []
for i in range(len(monthly_rainfall_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourO.append('L' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Rainfall']
for i in range(len(colourO)):
    ws[(colourO[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [226]:
colourP = []
for i in range(len(monthly_rainfall_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourP.append('M' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Rainfall']
for i in range(len(colourP)):
    ws[(colourP[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [227]:
colourQ = []
for i in range(len(monthly_rainfall_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourQ.append('N' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Rainfall']
for i in range(len(colourQ)):
    ws[(colourQ[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

###### Monthly Statistics

In [228]:
colourR = []
for i in range(len(monthly_stats_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourR.append('A' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Statistics']
for i in range(len(colourR)):
    ws[(colourR[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [229]:
colourS = []
for i in range(len(monthly_stats_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourS.append('B' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Statistics']
for i in range(len(colourS)):
    ws[(colourS[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [230]:
colourT = []
for i in range(len(monthly_stats_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourT.append('C' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Statistics']
for i in range(len(colourT)):
    ws[(colourT[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [231]:
colourU = []
for i in range(len(monthly_stats_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourU.append('D' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Statistics']
for i in range(len(colourU)):
    ws[(colourU[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [232]:
colourV = []
for i in range(len(monthly_stats_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourV.append('E' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Statistics']
for i in range(len(colourV)):
    ws[(colourV[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [233]:
colourW = []
for i in range(len(monthly_stats_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourW.append('F' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Statistics']
for i in range(len(colourW)):
    ws[(colourW[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

###### Seasonal Rainfall

In [234]:
colourY = []
for i in range(len(seasonal_rainfall_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourY.append('A' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Seasonal Rainfall']
for i in range(len(colourY)):
    ws[(colourY[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [235]:
colourZ = []
for i in range(len(seasonal_rainfall_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourZ.append('B' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Seasonal Rainfall']
for i in range(len(colourZ)):
    ws[(colourZ[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [236]:
colourAA = []
for i in range(len(seasonal_rainfall_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourAA.append('C' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Seasonal Rainfall']
for i in range(len(colourAA)):
    ws[(colourAA[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [237]:
colourAB = []
for i in range(len(seasonal_rainfall_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourAB.append('D' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Seasonal Rainfall']
for i in range(len(colourAB)):
    ws[(colourAB[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [238]:
colourAC = []
for i in range(len(seasonal_rainfall_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourAC.append('E' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Seasonal Rainfall']
for i in range(len(colourAC)):
    ws[(colourAC[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

###### Seasonal Statistics

In [239]:
colourAD = []
for i in range(len(seasonal_stats_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourAD.append('A' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Seasonal Statistics']
for i in range(len(colourAD)):
    ws[(colourAD[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [240]:
colourAE = []
for i in range(len(seasonal_stats_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourAE.append('B' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Seasonal Statistics']
for i in range(len(colourAE)):
    ws[(colourAE[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [241]:
colourAF = []
for i in range(len(seasonal_stats_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourAF.append('C' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Seasonal Statistics']
for i in range(len(colourAF)):
    ws[(colourAF[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [242]:
colourAG = []
for i in range(len(seasonal_stats_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourAG.append('D' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Seasonal Statistics']
for i in range(len(colourAG)):
    ws[(colourAG[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [243]:
colourAH = []
for i in range(len(seasonal_stats_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourAH.append('E' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Seasonal Statistics']
for i in range(len(colourAH)):
    ws[(colourAH[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [244]:
colourAI = []
for i in range(len(seasonal_stats_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourAI.append('F' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Seasonal Statistics']
for i in range(len(colourAI)):
    ws[(colourAI[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

###### Gumbel Estimate

In [245]:
colourAJ = []
for i in range(len(gumbel_est_calculations_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourAJ.append('A' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Gumbel Estimate']
for i in range(len(colourAJ)):
    ws[(colourAJ[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [246]:
colourAK = []
for i in range(len(gumbel_est_calculations_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourAK.append('B' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Gumbel Estimate']
for i in range(len(colourAK)):
    ws[(colourAK[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [247]:
colourAL = []
for i in range(len(gumbel_est_calculations_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourAL.append('C' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Gumbel Estimate']
for i in range(len(colourAL)):
    ws[(colourAL[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [248]:
colourAM = []
for i in range(len(gumbel_est_calculations_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourAM.append('D' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Gumbel Estimate']
for i in range(len(colourAM)):
    ws[(colourAM[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [249]:
colourAN = []
for i in range(len(gumbel_est_calculations_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourAN.append('E' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Gumbel Estimate']
for i in range(len(colourAN)):
    ws[(colourAN[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [250]:
colourAO = []
for i in range(len(gumbel_est_calculations_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourAO.append('F' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Gumbel Estimate']
for i in range(len(colourAO)):
    ws[(colourAO[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [251]:
colourAP = []
for i in range(len(gumbel_est_calculations_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourAP.append('G' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Gumbel Estimate']
for i in range(len(colourAP)):
    ws[(colourAP[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [252]:
colourAQ = []
for i in range(len(gumbel_est_calculations_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourAQ.append('H' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Gumbel Estimate']
for i in range(len(colourAQ)):
    ws[(colourAQ[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [253]:
colourAR = []
for i in range(len(gumbel_est_calculations_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourAR.append('I' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Gumbel Estimate']
for i in range(len(colourAR)):
    ws[(colourAR[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

###### Auckland Jan 2023 Stats

In [254]:
colourAS = []
for i in range(len(january_2023_stats_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourAS.append('A' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Auckland Jan 2023 Stats']
for i in range(len(colourAS)):
    ws[(colourAS[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [255]:
colourAT = []
for i in range(len(january_2023_stats_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourAT.append('B' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Auckland Jan 2023 Stats']
for i in range(len(colourAT)):
    ws[(colourAT[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

In [256]:
colourAU = []
for i in range(len(january_2023_stats_df)+2):
    if i % 2 == 0:
        if i != 0:
            colourAU.append('C' + str(i))
            
wb = load_workbook('output_file.xlsx')
ws = wb['Auckland Jan 2023 Stats']
for i in range(len(colourAU)):
    ws[(colourAU[i])].fill = PatternFill(start_color = 'FCCA88', end_color = 'FCCA88', fill_type = 'solid')
wb.save('output_file.xlsx')
wb.close()

### 5.3.4 Insert Excel Headers and Information Boxes (.png)

###### Auckland Aero Rainfall Data

In [257]:
wb = load_workbook('output_file.xlsx')
ws = wb['Auckland Aero Rainfall Data']

img = openpyxl.drawing.image.Image('info_headers/info_headers (1).png')
img.width = 640
img.height = 345
img.anchor = 'H1'
ws.add_image(img)

wb.save('output_file.xlsx')
wb.close()

###### Monthly Rainfall

In [258]:
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Rainfall']

img = openpyxl.drawing.image.Image('info_headers/info_headers (2).png')
img.width = 640
img.height = 345
img.anchor = 'S1'
ws.add_image(img)

wb.save('output_file.xlsx')
wb.close()

###### Monthly Statistics

In [259]:
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Statistics']

img = openpyxl.drawing.image.Image('info_headers/info_headers (3).png')
img.width = 640
img.height = 345
img.anchor = 'K1'
ws.add_image(img)

wb.save('output_file.xlsx')
wb.close()

###### Seasonal Rainfall

In [260]:
wb = load_workbook('output_file.xlsx')
ws = wb['Seasonal Rainfall']

img = openpyxl.drawing.image.Image('info_headers/info_headers (4).png')
img.width = 640
img.height = 345
img.anchor = 'J1'
ws.add_image(img)

wb.save('output_file.xlsx')
wb.close()

###### Seasonal Statistics

In [261]:
wb = load_workbook('output_file.xlsx')
ws = wb['Seasonal Statistics']

img = openpyxl.drawing.image.Image('info_headers/info_headers (5).png')
img.width = 640
img.height = 345
img.anchor = 'K1'
ws.add_image(img)

wb.save('output_file.xlsx')
wb.close()

###### Gumbel Estimate

In [262]:
wb = load_workbook('output_file.xlsx')
ws = wb['Gumbel Estimate']

img = openpyxl.drawing.image.Image('info_headers/info_headers (6).png')
img.width = 640
img.height = 345
img.anchor = 'N1'
ws.add_image(img)

wb.save('output_file.xlsx')
wb.close()

In [263]:
wb = load_workbook('output_file.xlsx')
ws = wb['Gumbel Estimate']

img = openpyxl.drawing.image.Image('gumbel_variables.png')
img.width = 450
img.height = 225
img.anchor = 'Q43'
ws.add_image(img)

wb.save('output_file.xlsx')
wb.close()

In [264]:
wb = load_workbook('output_file.xlsx')
ws = wb['Gumbel Estimate']

img = openpyxl.drawing.image.Image('gumbel_est.png')
img.width = 700
img.height = 400
img.anchor = 'M57'
ws.add_image(img)

wb.save('output_file.xlsx')
wb.close()

###### Auckland Jan 2023 Stats

In [265]:
wb = load_workbook('output_file.xlsx')
ws = wb['Auckland Jan 2023 Stats']

img = openpyxl.drawing.image.Image('info_headers/info_headers (7).png')
img.width = 640
img.height = 345
img.anchor = 'H1'
ws.add_image(img)

wb.save('output_file.xlsx')
wb.close()

**************************************************************************************************

## 5.4 Add Graphs to Excel Spreadsheet

###### Auckland Aero Rainfall Data

In [266]:
a = len(rainfall_data_df)+1

In [267]:
wb = load_workbook('output_file.xlsx')
ws = wb['Auckland Aero Rainfall Data']

# Data for plotting
y_values = Reference(ws, min_col=3, max_col=3, min_row=3, max_row=a)
x_values = Reference(ws, min_col=2, max_col=2, min_row=2, max_row=a)

# Create object of BarChart class
chart = BarChart()
chart.style = 3 #Style Modification
chart.add_data(y_values)
chart.set_categories(x_values)

# set the title of the chart
chart.title = 'Auckland Areo Rainfall (mm/d) (1962/07/01 - 2023/06/29)'

# set the title of the x-axis
chart.x_axis.title = 'Date'

# set the title of the y-axis
chart.y_axis.title = 'Rainfall (mm/d)'

# Change bar filling and line color 
s = chart.series[0]
s.graphicalProperties.line.solidFill = 'D1831A'
s.graphicalProperties.solidFill = 'D1831A'

# Change bar size 
chart.height = 11  # default is 7.5
chart.width = 22  # default is 15

#Delete Legend
chart.legend = None

# the top-left corner of the chart
ws.add_chart(chart,"E20")

wb.save('output_file.xlsx')
wb.close()

###### Monthly Rainfall

In [268]:
trace0 = go.Box(
    y = monthly_rainfall_df['January'],
    name = 'January'
)
trace1 = go.Box(
    y = monthly_rainfall_df['February'],
    name = 'February'
)
trace2 = go.Box(
    y = monthly_rainfall_df['March'],
    name = 'March'
)
trace3 = go.Box(
    y = monthly_rainfall_df['April'],
    name = 'April'
)
trace4 = go.Box(
    y = monthly_rainfall_df['May'],
    name = 'May'
)
trace5 = go.Box(
    y = monthly_rainfall_df['June'],
    name = 'June'
)
trace6 = go.Box(
    y = monthly_rainfall_df['July'],
    name = 'July'
)
trace7 = go.Box(
    y = monthly_rainfall_df['August'],
    name = 'August'
)
trace8 = go.Box(
    y = monthly_rainfall_df['September'],
    name = 'September'
)
trace9 = go.Box(
    y = monthly_rainfall_df['October'],
    name = 'October'
)
trace10 = go.Box(
    y = monthly_rainfall_df['November'],
    name = 'November'
)
trace11 = go.Box(
    y = monthly_rainfall_df['December'],
    name = 'December'
)

data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9, trace10, trace11]
layout = go.Layout(title = 'Box and Whisker: Monthly Rainfall (mm)')

fig = go.Figure(data=data, layout=layout)
pyo.plot(fig)

img_bytes = fig.to_image(format="png", width=832, height=567, scale=2)
Image(img_bytes)
fig.write_image('monthly_box_whisker.png')

In [269]:
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Rainfall']

img = openpyxl.drawing.image.Image('monthly_box_whisker.png')
img.width = 832
img.height = 567
img.anchor = 'P20'
ws.add_image(img)

wb.save('output_file.xlsx')
wb.close()

###### Monthly Statistics

In [270]:
c = len(monthly_stats_df)+1

In [271]:
wb = load_workbook('output_file.xlsx')
ws = wb['Monthly Statistics']

# Data for plotting
y_values = Reference(ws, min_col=2, max_col=6, min_row=1, max_row=c)
x_values = Reference(ws, min_col=1, max_col=1, min_row=2, max_row=c)

# Create object of LineChart class
chart = LineChart()
chart.style = 3 #Style Modification
chart.add_data(data=y_values, titles_from_data=True)
chart.set_categories(x_values)

# set the title of the chart
chart.title = 'Monthly Statistics'

# set the title of the x-axis
chart.x_axis.title = 'Month'

# set the title of the y-axis
chart.y_axis.title = 'Rainfall (mm)'

# Change bar filling and line color 
s = chart.series[0]
s.graphicalProperties.line.solidFill = 'D1831A'
s.graphicalProperties.solidFill = 'D1831A'

# Change bar size 
chart.height = 11  # default is 7.5
chart.width = 22  # default is 15

# the top-left corner of the chart
ws.add_chart(chart,"H20")

wb.save('output_file.xlsx')
wb.close()

time.sleep(5)

###### Seasonal Rainfall

In [272]:
trace0 = go.Box(
    y = seasonal_rainfall_df['Summer (Dec - Feb)'],
    name = 'Summer (Dec - Feb)'
)
trace1 = go.Box(
    y = seasonal_rainfall_df['Autumn (March - May)'],
    name = 'Autumn (March - May)'
)
trace2 = go.Box(
    y = seasonal_rainfall_df['Winter (Jun - Aug)'],
    name = 'Winter (Jun - Aug)'
)
trace3 = go.Box(
    y = seasonal_rainfall_df['Spring (Sep - Nov)'],
    name = 'Spring (Sep - Nov)'
)

data = [trace0, trace1, trace2, trace3]
layout = go.Layout(title = 'Box and Whisker: Seasonal Rainfall (mm)')

fig = go.Figure(data=data, layout=layout)
pyo.plot(fig)

img_bytes = fig.to_image(format="png", width=832, height=567, scale=2)
Image(img_bytes)
fig.write_image('seasonal_box_whisker.png')

In [273]:
wb = load_workbook('output_file.xlsx')
ws = wb['Seasonal Rainfall']

img = openpyxl.drawing.image.Image('seasonal_box_whisker.png')
img.width = 832
img.height = 567
img.anchor = 'G20'
ws.add_image(img)

wb.save('output_file.xlsx')
wb.close()

###### Seasonal Statistics

In [274]:
e = len(seasonal_stats_df)+1

In [275]:
wb = load_workbook('output_file.xlsx')
ws = wb['Seasonal Statistics']

# Data for plotting
y_values = Reference(ws, min_col=2, max_col=6, min_row=1, max_row=e)
x_values = Reference(ws, min_col=1, max_col=1, min_row=2, max_row=e)

# Create object of LineChart class
chart = LineChart()
chart.style = 3 #Style Modification
chart.add_data(data=y_values, titles_from_data=True)
chart.set_categories(x_values)

# set the title of the chart
chart.title = 'Seasonal Statistics'

# set the title of the x-axis
chart.x_axis.title = 'Seasons'

# set the title of the y-axis
chart.y_axis.title = 'Rainfall (mm)'

# Change bar filling and line color 
s = chart.series[0]
s.graphicalProperties.line.solidFill = 'D1831A'
s.graphicalProperties.solidFill = 'D1831A'

# Change bar size 
chart.height = 11  # default is 7.5
chart.width = 22  # default is 15

# the top-left corner of the chart
ws.add_chart(chart,"H20")

wb.save('output_file.xlsx')
wb.close()

time.sleep(5)

###### Gumbel Estimate

In [276]:
g = len(gumbel_est_calculations_df)+1

In [277]:
wb = load_workbook('output_file.xlsx')
ws = wb['Gumbel Estimate']

# Data for plotting
y_values = Reference(ws, min_col=7, max_col=7, min_row=1, max_row=g)
x_values = Reference(ws, min_col=1, max_col=1, min_row=2, max_row=g)

# Create object of BarChart class
chart = BarChart()
chart.style = 3 #Style Modification
chart.add_data(data=y_values, titles_from_data=True)
chart.set_categories(x_values)

# set the title of the chart
chart.title = 'Return Period'

# set the title of the x-axis
chart.x_axis.title = 'Year'

# set the title of the y-axis
chart.y_axis.title = 'Return Period'

# Change bar filling and line color 
s = chart.series[0]
s.graphicalProperties.line.solidFill = 'D1831A'
s.graphicalProperties.solidFill = 'D1831A'

# Change bar size 
chart.height = 11  # default is 7.5
chart.width = 22  # default is 15

# the top-left corner of the chart
ws.add_chart(chart,"K20")

wb.save('output_file.xlsx')
wb.close()

time.sleep(5)

###### Auckland Jan 2023 Stats

In [278]:
h = len(january_2023_stats_df)+1

In [279]:
wb = load_workbook('output_file.xlsx')
ws = wb['Auckland Jan 2023 Stats']

# Data for plotting
y_values = Reference(ws, min_col=2, max_col=3, min_row=1, max_row=h)
x_values = Reference(ws, min_col=1, max_col=1, min_row=2, max_row=h)

# Create object of LineChart class
chart = LineChart()
chart.style = 3 #Style Modification
chart.add_data(data=y_values, titles_from_data=True)
chart.set_categories(x_values)

# set the title of the chart
chart.title = 'January (1963 - 2022) vs. January 2023'

# set the title of the x-axis
chart.x_axis.title = 'Statistic Description'

# set the title of the y-axis
chart.y_axis.title = 'Rainfall (mm)'

# Change bar filling and line color 
s = chart.series[0]
s.graphicalProperties.line.solidFill = 'D1831A'
s.graphicalProperties.solidFill = 'D1831A'

# Change bar size 
chart.height = 11  # default is 7.5
chart.width = 22  # default is 15

# the top-left corner of the chart
ws.add_chart(chart,"E20")

wb.save('output_file.xlsx')
wb.close()

time.sleep(5)

**************************************************************************************************

## 5.5 Save Excel to Folder

In [280]:
os.remove('seasonal_box_whisker.png')
os.remove('temp-plot.html')
os.remove('monthly_box_whisker.png')
os.remove('gumbel_est.png')
os.remove('gumbel_variables.png')

**************************************************************************************************
**************************************************************************************************